# 0. Importing data 

In [3]:
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from sklearn import svm
import xgboost as xgb
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
from nltk import *
import os
import statistics
import editdistance
import itertools  
import re



In [4]:
cl = sklearn.linear_model.Perceptron()

In [7]:
full_train_df = pd.read_csv('quora_question_pairs/quora_train.csv')

# 1. Raw data cleaning and preprocessing 

In [8]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings


Use all the questions in train and test partitions to build a single list `all_questions` to fit the `count_vectorizer`

### Remove Punctuation, substituting and lowering
Stolen from https://www.kaggle.com/benjaminkz/quora-question-pairs-data-cleaning

In [9]:
def common_words_transformation_remove_punctuation(text):
    
    text = text.lower()
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"when's", "when is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"there's", "there is", text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)  # 除了上面的特殊情况外，“\'s”只能表示所有格，应替换成“ ”
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " america ", text)
    text = re.sub(r" u s ", " america ", text)
    text = re.sub(r" uk ", " england ", text)
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text)
    text = re.sub(r" ds ", " data science ", text)
    text = re.sub(r" ee ", " electronic engineering ", text)
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iphone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the us", "america", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    
    punctuation="?:!.,;"

    text = "".join([c for c in text if c not in punctuation])
    
    return text

In [10]:
doc = full_train_df['question1'][0]
doc

'What is the step by step guide to invest in share market in india?'

In [11]:
doc = common_words_transformation_remove_punctuation(doc)
doc

'what is the step by step guide to invest in share market in india '

### Removing Stop_Words

In [13]:
def remove_stopwords(doc):
    stops     = set(stopwords.words("english"))
    token_doc = set(word_tokenize(doc))
    token_doc = list(token_doc - stops)
    return " ".join(token_doc)

In [14]:
print(doc)
doc = remove_stopwords(doc)

what is the step by step guide to invest in share market in india 


### Stemming

In [15]:
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer

porter    = PorterStemmer()
#lancaster = LancasterStemmer()

In [16]:
from nltk.tokenize import sent_tokenize, word_tokenize

def stem(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(porter.stem(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

In [17]:
print(doc)
doc = stem(doc)

india invest share step guide market


### Lemmatization

In [18]:
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()



In [19]:
def lemm(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(wordnet_lemmatizer.lemmatize(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

In [20]:
print(doc)
doc = stem(doc)

india invest share step guid market 


### Putting all together

In [21]:
def preprocess(df, save=False):
    df = df.copy()
    df['question1'] = cast_list_as_strings(list(df["question1"]))
    df['question2'] = cast_list_as_strings(list(df["question2"]))
        
    df['question1_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question1"]]
    df['question2_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question2"]]
        
    df['question1_no_stops'] = [remove_stopwords(quest) for quest in df["question1_cleaned"]]
    df['question2_no_stops'] = [remove_stopwords(quest) for quest in df["question2_cleaned"]]
    
    df['question1_Porter_stemmed'] = [stem(quest) for quest in df["question1_cleaned"]]
    df['question2_Porter_stemmed'] = [stem(quest) for quest in df["question2_cleaned"]]
        
    df['question1_Lemmatization']  = [lemm(quest) for quest in df["question1_cleaned"]]
    df['question2_Lemmatization']  = [lemm(quest) for quest in df["question2_cleaned"]]
    
    if save==True:
        df.to_csv('preprocessed_Quora_full_train_data.csv')
        
    return df

In [22]:
full_train_proc = preprocess(full_train_df, save=True)

In [23]:
full_train_proc.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_cleaned,question2_cleaned,question1_no_stops,question2_no_stops,question1_Porter_stemmed,question2_Porter_stemmed,question1_Lemmatization,question2_Lemmatization
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,india invest share step guide market,invest share step guide market,what is the step by step guid to invest in sha...,what is the step by step guid to invest in sha...,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,koh noor diamond kohinoor story,stole government indian happen koh back noor d...,what is the stori of kohinoor koh i noor diamond,what would happen if the indian govern stole t...,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,speed internet using increase vpn connection,hacking speed increased internet dns,how can i increas the speed of my internet con...,how can internet speed be increas by hack thro...,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math is d...,lonely solve mentally,24 remainder 23 divided find math,whi am i mental veri lone how can i solv it,find the remaind when math 23 24 math is divid...,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math is div...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,which one dissolve in water quickly sugar sal...,which fish would survive in salt water,sugar methane quickly water one salt carbon ox...,fish water salt survive would,which one dissolv in water quickli sugar salt ...,which fish would surviv in salt water,which one dissolve in water quickly sugar salt...,which fish would survive in salt water


## Adjusting empty questions
There are some questions that are empty. We drop them.

In [24]:
def cont_len(question):
    num_words = len(question.split())
    return num_words

#Dropping all the sample where there is not one of the two questions
len_1   = full_train_proc.question1_cleaned.apply(lambda x: cont_len(x))
empty_1 = len_1[len_1==0].index

len_2   = full_train_proc.question2_cleaned.apply(lambda x: cont_len(x))
empty_2 = len_2[len_2==0].index

to_delete = (empty_2).union(set(empty_1))

full_train_proc = full_train_proc.drop(index=to_delete)

# 2. Other Features Vectors

### These functions generate new feature in the df

In [31]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jsreu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [25]:
def len_common(q1, q2):
    """
    counts number of words in common
    """
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1.intersection(q2))

def len_not_common(q1,q2):
    """
    counts number of words not in common
    """
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1 ^ q2)

#Adding mean distance between common words 
def mean_dist_not_com(q1,q2):
    """
    mean edit distance between words not in common
    """
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    not_comm1 = (q1 ^ q2) - q1
    if len(not_comm1)==0 : not_comm1={''}
    not_comm2 = (q1 ^ q2) - q2
    if len(not_comm2)==0 : not_comm2={''}
    return statistics.mean([editdistance.eval(i[0],i[1]) for i in itertools.product(not_comm1, not_comm2)])

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString)) 
def both_number(q1,q2):
    """
    If in both questions ther is a number
    """
    return hasNumbers(q1) *  hasNumbers(q2) 

def average_len(question):
    """
    get average number of words
    """
    num_words = len(question.split())
    return len(question)/num_words   

def levenshtein(q1, q2): 
    """
    levenshtein distance (for strings of unequal length)
    """
    #create initial array (two for loops since q1 and q2 can differ in length)
    dist_array = []
    for i in range(len(q1)+1):
        dist_array.append([0]*(len(q2)+1))
        dist_array[i][0] = i
    for j in range(len(q2)+1):
        dist_array[0][j] = j

    dist = [0]*3
    for i in range(1,len(q1)+1):
        for j in range(1,len(q2)+1):
            dist[0] = dist_array[i-1][j-1] if q1[i-1]==q2[j-1] else dist_array[i-1][j-1]+1
            dist[1] = dist_array[i][j-1]+1
            dist[2] = dist_array[i-1][j]+1
            dist_array[i][j]=min(dist)
    
    return dist_array[i][j]

#from math import*
def jaccard_similarity(row):
    q1 = row["question1"]
    q2 = row["question2"]
    set1 = set(q1.split()) 
    set2 = set(q2.split())
    same = set1.intersection(set2)
    return float(len(same)) / (len(set1) + len(set2) - len(same))

def common_tokens(string_1,string_2):
    """
    counts common word types. options are NOUN,VERB,ADV,ADJ
    """
    string_1 = str(string_1)
    string_2 = str(string_2)
    
    x = nltk.word_tokenize(string_1)
    y = nltk.word_tokenize(string_2)   
 
    common_tokens = len(list(set(x).intersection(y)))
    
    return(common_tokens)

def common_count(string_1,string_2, word_type = "NOUN"):
    """
    counts common word types. options are NOUN,VERB,ADV,ADJ
    """
    string_1 = str(string_1)
    string_2 = str(string_2)
    
    tagged_1 = nltk.pos_tag(nltk.word_tokenize(string_1),tagset='universal')
    tagged_2 = nltk.pos_tag(nltk.word_tokenize(string_2),tagset='universal')
    
    x = list([])
    for word in tagged_1:
        if word[1] == word_type:
            x.append(word[0])
                     
    y = list([])
    for word in tagged_2:
        if word[1] == word_type:
            y.append(word[0])
 
    common = len(list(set(x).intersection(y)))
    
    return(common)

def length_diff_characters(x,y):
    """
    find absolute difference in character length of two questions
    """
    x = str(x)
    y = str(y)
    return abs(len(y)-len(x))

def length_diff_tokens(x,y):
    """
    find absolute difference in number of tokens in two questions
    """
    return abs(len(nltk.word_tokenize(y))-len(nltk.word_tokenize(x)))

def common_numbers(x,y):
    """
    count numbers present in both tokens
    """
    x_numbers = re.findall(r'\b\d+\b', x)
    y_numbers = re.findall(r'\b\d+\b', y)
    
    common_numbers = len(list(set(x_numbers).intersection(y_numbers)))
    return(common_numbers)

def first_word(x,y):
    """
    boolean if the first word is the same
    """
    word_list_x = x.split()  # list of words
    word_list_y = y.split()  # list of words
    return word_list_x[0] == word_list_y[0]

def last_word(x,y):
    """
    boolean if the last word is the same
    """
    word_list_x = x.split()  # list of words
    word_list_y = y.split()  # list of words
    return word_list_x[-1] == word_list_y[-1]


### Add_features() apply to the data all the feature generator functions

In [29]:
def Add_features(df, scaler, col1, col2, save=False):
    df = df.copy()
    col1_to_transform = list(full_train_proc.columns).index(col1)
    col2_to_transform = list(full_train_proc.columns).index(col2)

    df['len_q1'] = [len(s) for s in df[col1]] 
    df['len_q2'] = [len(s) for s in df[col2]]
    df['len_q1'] = scaler.fit_transform(df[['len_q1']])
    df['len_q2'] = scaler.fit_transform(df[['len_q2']])
    
    df['len_common'] = [len_common(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['len_common'] = scaler.fit_transform(df[['len_common']])

    df['len_not_common'] = [len_not_common(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['len_not_common'] = scaler.fit_transform(df[['len_not_common']])
    
    df['mean_dist_not_com'] = [mean_dist_not_com(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['mean_dist_not_com'] = scaler.fit_transform(df[['mean_dist_not_com']])
    
    df['both_number'] = [both_number(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    
    df["avg_len_q1"]= df[col1].apply(lambda x: average_len(x))
    df["avg_len_q2"]= df[col2].apply(lambda x: average_len(x))
    df['avg_len_q1'] = scaler.fit_transform(df[['avg_len_q1']])
    df['avg_len_q2'] = scaler.fit_transform(df[['avg_len_q2']])

    df['edit_distance'] = [editdistance.eval(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['edit_distance'] = scaler.fit_transform(df[['edit_distance']])
    
    df['levenshtein_distance'] = [levenshtein(df.iloc[i,col1_to_transform],df.iloc[i,col2_to_transform]) for i in range(len(df))]
    df['levenshtein_distance'] = scaler.fit_transform(df[['levenshtein_distance']])
    
    df["common_nouns"] = df.apply(lambda df: common_count(df[col1], df[col2],"NOUN"), axis=1)
    df['common_nouns'] = scaler.fit_transform(df[['common_nouns']])

    df["common_verbs"] = df.apply(lambda df: common_count(df[col1], df[col2],"VERB"), axis=1)
    df['common_verbs'] = scaler.fit_transform(df[['common_verbs']])
    
    df["common_adjectives"] = df.apply(lambda df: common_count(df[col1], df[col2],"ADJ"), axis=1)
    df['common_adjectives'] = scaler.fit_transform(df[['common_adjectives']])
    
    df["common_adverbs"] = df.apply(lambda df: common_count(df[col1], df[col2],"ADV"), axis=1)
    df['common_adverbs'] = scaler.fit_transform(df[['common_adverbs']])

    df["common_tokens"] = df.apply(lambda df: common_tokens(df[col1], df[col2]), axis=1)
    df['common_tokens'] = scaler.fit_transform(df[['common_tokens']])

    df["length_diff_characters"] = df.apply(lambda df: length_diff_characters(df[col1], df[col2]), axis=1)
    df['length_diff_characters'] = scaler.fit_transform(df[['length_diff_characters']])

    df["length_diff_tokens"] = df.apply(lambda df: length_diff_tokens(df[col1], df[col2]), axis=1)
    df['length_diff_tokens'] = scaler.fit_transform(df[['length_diff_tokens']])

    df["common_numbers"] = df.apply(lambda df: length_diff_characters(df.question1, df.question2), axis=1)
    df['common_numbers'] = scaler.fit_transform(df[['common_numbers']])

    df['first_word'] = df.apply(lambda df: first_word(df[col1], df[col2]), axis=1)*1
    df['last_word'] = df.apply(lambda df: last_word(df[col1], df[col2]), axis=1)*1
    
    df["jaccard_similarity"] = df.apply(lambda x: jaccard_similarity(x), axis=1)
    df['jaccard_similarity'] = scaler.fit_transform(df[['jaccard_similarity']])
    
    if save==True:
        df.to_csv('features_added_Quora_full_train_data.csv')
    
    return df

In [32]:
scaler = MinMaxScaler()

full_train_proc_plus_feat = Add_features(full_train_proc, scaler, 'question1_cleaned', 'question2_cleaned', save=True)


# 3. Two Simple Models

In [33]:
def fit_on_q1_q2(df, model):
    q_list1 = list(df["question1_cleaned"])
    q_list2 = list(df["question2_cleaned"])
    all_questions = q_list1 + q_list2 
    model.fit(all_questions)
    return

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    q2_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2

Training a Logistic and a XGB on the cleaned questions using BOW

In [66]:
def train_models(df, get_feat_model, vectorizer_func):
    
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, 
                              learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
                              objective='binary:logistic', eta=0.3, silent=1, subsample=0.8, random_state=123)

    #svm_model = svm.SVC()
    
    train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_feat_model(train_df, vectorizer_func)
    X_val_q1q2        = get_feat_model(val_df, vectorizer_func)
    X_test_q1q2       = get_feat_model(test_df, vectorizer_func)
    
    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)
    xgb_model.fit(X_train_q1q2, y_train) 
    #svm_model.fit(X_tr_q1q2, y_train)
                                                   
    logistic_train_acc = roc_auc_score(y_train, logistic.predict_proba(X_train_q1q2)[:, 1])                                                       
    logistic_val_acc   = roc_auc_score(y_val, logistic.predict_proba(X_val_q1q2)[:, 1])
    logistic_test_acc  = roc_auc_score(y_test, logistic.predict_proba(X_test_q1q2)[:, 1])
    print('logistic_train_acc:{}, logistic_val_acc:{}, logistic_test_acc:{}'.format(logistic_train_acc, logistic_val_acc, logistic_test_acc))
                                                   
    xgb_train_acc      = roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])
    xgb_val_acc        = roc_auc_score(y_val, xgb_model.predict_proba(X_val_q1q2)[:, 1])
    xgb_test_acc       = roc_auc_score(y_test, xgb_model.predict_proba(X_test_q1q2)[:, 1])
    print('xgb_train_acc:{}, xgb_val_acc:{}, xgb_test_acc:{}'.format(xgb_train_acc, xgb_val_acc, xgb_test_acc))
        
                                                   
    return logistic, xgb_model, [logistic_train_acc, logistic_val_acc, logistic_test_acc], [xgb_train_acc, xgb_val_acc, xgb_test_acc]
                                              
                                                       
                                                       
           

In [36]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

fit_on_q1_q2(full_train_proc_plus_feat, count_vectorizer)

In [179]:
logistic_simple, xgb_model_simple, acc_logistic_simple, acc_xgboost_simple = \
train_models(full_train_proc_plus_feat, get_features_from_df, count_vectorizer)


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:45:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:45:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8560015673032263, logistic_val_acc:0.7953934745032775, logistic_test_acc:0.7956985843493918
xgb_train_acc:0.8659191549450259, xgb_val_acc:0.8093737107889942, xgb_test_acc:0.8097783288425393


Training a Logistic and a XGB on the cleaned questions using tfidf

In [180]:
def get_tfidf(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    q2_casted =  cast_list_as_strings(list(df["question1_cleaned"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [181]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(full_train_proc_plus_feat, tfidf)

In [182]:
logistic_simple_tf, xgb_model_simple_tf, acc_logistic_simple_tf, acc_xgboost_simple_tf = \
train_models(full_train_proc_plus_feat, get_tfidf, tfidf)


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:58:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:59:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8416865189699401, logistic_val_acc:0.7958238445455491, logistic_test_acc:0.797239576925177
xgb_train_acc:0.9070173792036841, xgb_val_acc:0.8234103363606295, xgb_test_acc:0.827242512987826


# 4. More advanced models

## A. Adding Lemmatization/Stemming and removing stop words

Training a Logistic and a XGB on the cleaned questions using tfidf and Lemmatization

In [62]:
full_train_proc_plus_feat = full_train_proc.copy()

In [63]:
def get_tfidf_lemma(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1_Lemmatization"]))
    q2_casted =  cast_list_as_strings(list(df["question2_Lemmatization"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [64]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(full_train_proc_plus_feat, tfidf)

In [65]:
logistic_simple_tf, xgb_model_simple_tf, acc_logistic_simple_tf, acc_xgboost_simple_tf = \
train_models(full_train_proc_plus_feat, get_tfidf_lemma, tfidf)

C:\Users\jsreu\anaconda3\envs\NLP\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:25:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:26:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.855750814661588, logistic_val_acc:0.8068484115751718, logistic_test_acc:0.8085147242608809
xgb_train_acc:0.9647508218153511, xgb_val_acc:0.8711220416076965, xgb_test_acc:0.8764785359835098


Training a Logistic and a XGB on the cleaned questions using tfidf and Stemming

In [67]:
full_train_proc_plus_feat = full_train_proc.copy()

In [68]:
def get_tfidf_stem(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1_Porter_stemmed"]))
    q2_casted =  cast_list_as_strings(list(df["question2_Porter_stemmed"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [69]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(full_train_proc_plus_feat, tfidf)

In [70]:
logistic_simple_tf, xgb_model_simple_tf, acc_logistic_simple_tf, acc_xgboost_simple_tf = \
train_models(full_train_proc_plus_feat, get_tfidf_stem, tfidf)

C:\Users\jsreu\anaconda3\envs\NLP\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:46:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:46:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8198517225646291, logistic_val_acc:0.7864257782597975, logistic_test_acc:0.7861417455061237
xgb_train_acc:0.961030138328266, xgb_val_acc:0.8669393473334588, xgb_test_acc:0.8700238109795451


Training a Logistic and a XGB on the cleaned questions using tfidf and removing stopwords

In [57]:
full_train_proc_plus_feat = full_train_proc.copy()

In [58]:
def get_tfidf_stops(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1_no_stops"]))
    q2_casted =  cast_list_as_strings(list(df["question2_no_stops"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [59]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(full_train_proc_plus_feat, tfidf)

In [60]:
logistic_simple_tf, xgb_model_simple_tf, acc_logistic_simple_tf, acc_xgboost_simple_tf = \
train_models(full_train_proc_plus_feat, get_tfidf_stops, tfidf)

C:\Users\jsreu\anaconda3\envs\NLP\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:12:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:12:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8597280270805987, logistic_val_acc:0.7999421615386503, logistic_test_acc:0.8012380254227004
xgb_train_acc:0.881222092059111, xgb_val_acc:0.8283344708581338, xgb_test_acc:0.8343091186477951


## B. Adding additional features

### Adding cosine similarity

In [190]:
def get_tfidf(df, tfidf, sim=True):
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 
    
    

In [191]:
logistic_simple_tf_sim, xgb_model_simple_tf_sim, acc_logistic_tf_sim, acc_xgboost_tf_sim = \
train_models(full_train_proc_plus_feat, get_tfidf, tfidf)


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:56:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:56:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.9074969597126898, logistic_val_acc:0.8726778599886169, logistic_test_acc:0.8772466341269656
xgb_train_acc:0.9766716766767485, xgb_val_acc:0.8987541590750521, xgb_test_acc:0.9019259149390173


### Adding features based on length

In [303]:
def vectorize_train_val_test(df, get_feat_model, vectorizer_func):
    
    train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_feat_model(train_df, vectorizer_func)
    X_val_q1q2        = get_feat_model(val_df, vectorizer_func)
    X_test_q1q2       = get_feat_model(test_df, vectorizer_func)
    
    return  train_df, val_df, test_df, X_train_q1q2, X_val_q1q2, X_test_q1q2

In [306]:
def add_a_column_feat(data, col, sparse_matrix):
    feat_q = data[col].to_numpy().reshape(len(data[col]),1)
        
    return scipy.sparse.hstack((sparse_matrix,feat_q)).tocsr()        

def train_models_plus_feat(X_train_q1q2, X_val_q1q2, X_test_q1q2, train_df, val_df, test_df, col_list):    
    
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, 
                              learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
                              objective='binary:logistic', eta=0.3, silent=1, subsample=0.8, random_state=123)

    
    for col in col_list:
        X_train_q1q2      = add_a_column_feat(train_df, col, X_train_q1q2)
        X_val_q1q2        = add_a_column_feat(val_df, col, X_val_q1q2)
        X_test_q1q2       = add_a_column_feat(test_df, col, X_test_q1q2)
    print('...features added...')

    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)
    print('...logistic model fitted...')

    xgb_model.fit(X_train_q1q2, y_train) 
    print('...model fitted...')

    logistic_train_acc = roc_auc_score(y_train, logistic.predict_proba(X_train_q1q2)[:, 1])                                                       
    logistic_val_acc   = roc_auc_score(y_val, logistic.predict_proba(X_val_q1q2)[:, 1])
    logistic_test_acc  = roc_auc_score(y_test, logistic.predict_proba(X_test_q1q2)[:, 1])
    print('logistic_train_acc:{}, logistic_val_acc:{}, logistic_test_acc:{}'.format(logistic_train_acc, logistic_val_acc, logistic_test_acc))
                                                   
    xgb_train_acc      = roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])
    xgb_val_acc        = roc_auc_score(y_val, xgb_model.predict_proba(X_val_q1q2)[:, 1])
    xgb_test_acc       = roc_auc_score(y_test, xgb_model.predict_proba(X_test_q1q2)[:, 1])
    print('xgb_train_acc:{}, xgb_val_acc:{}, xgb_test_acc:{}'.format(xgb_train_acc, xgb_val_acc, xgb_test_acc))
                                                   
    return logistic, xgb_model, [logistic_train_acc, logistic_val_acc, logistic_test_acc], [xgb_train_acc, xgb_val_acc, xgb_test_acc]

    

In [ ]:
train_df, val_df, test_df, X_train_q1q2, X_val_q1q2, X_test_q1q2 = vectorize_train_val_test(full_train_proc_plus_feat, get_tfidf, tfidf)


In [301]:
cols_lenght = ['len_q1', 'len_q2', 'len_common', 'len_not_common', 'avg_len_q1', 'avg_len_q2']
logistic_simple_tf_sim_len, xgb_model_simple_tf_sim_len, acc_logistic_tfidf_sim_len, acc_xgboost_tfidf_sim_len \
 = train_models_plus_feat(X_train_q1q2, X_val_q1q2, X_test_q1q2, train_df, val_df, test_df, cols_lenght)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:09:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:10:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.9105569629864025, logistic_val_acc:0.877148443480405, logistic_test_acc:0.8810641865271815
xgb_train_acc:0.9893273664276654, xgb_val_acc:0.9060839423181429, xgb_test_acc:0.906456667632005


### Adding features based on distance

In [304]:
train_df, val_df, test_df, X_train_q1q2, X_val_q1q2, X_test_q1q2 = vectorize_train_val_test(full_train_proc_plus_feat, get_tfidf, tfidf)


In [309]:
cols_dist = cols_lenght + ['mean_dist_not_com', 'edit_distance']

logistic_simple_tf_sim_len_dist, xgb_model_simple_tf_sim_len_dist, acc_logistic_tfidf_sim_len_dist, acc_xgboost_tfidf_sim_len_dist \
 = train_models_plus_feat(X_train_q1q2, X_val_q1q2, X_test_q1q2, train_df, val_df, test_df, cols_dist)


...features added...
...logistic model fitted...
[22:45:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:45:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
...model fitted...
logistic_train_acc:0.9122547681146612, logistic_val_acc:0.879432834809194, logistic_test_acc:0.8829894326066793
xgb_train_acc:0.991570516727775, xgb_val_acc:0.9099064872805638, xgb_test_acc:0.9095038783787787


###  Adding remaining features

In [347]:
all_cols = cols_dist + ['both_number', 'first_word', 'last_word', 'length_diff_characters', 'length_diff_tokens', 'common_numbers']

logistic_simple_tf_sim_len_dist_extra, xgb_model_simple_tf_sim_len_dist_extra, acc_logistic_tfidf_sim_len_dist_extra, acc_xgboost_tfidf_sim_len_dist_extra \
 = train_models_plus_feat(X_train_q1q2, X_val_q1q2, X_test_q1q2, train_df, val_df, test_df, all_cols)


...features added...
...logistic model fitted...


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:50:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:51:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
...model fitted...
logistic_train_acc:0.9160962845200364, logistic_val_acc:0.8843373353169895, logistic_test_acc:0.8881570223833523
xgb_train_acc:0.9933322981345474, xgb_val_acc:0.9168239801135734, xgb_test_acc:0.9158050435110939


In [348]:
hist = pd.DataFrame({'BOW_No_Feat_Logistic':acc_logistic_simple,'BOW_No_Feat_XGB':acc_xgboost_simple,
                     'TFIDF_No_Feat_Logistic':acc_logistic_simple_tf,'TFIDF_No_Feat_XGB':acc_xgboost_simple_tf,
                     'TFIDF_Cos_Sim_Logistic':acc_logistic_tf_sim,'TFIDF_Cos_Sim_XGB':acc_xgboost_tf_sim,
                     'TFIDF_Cos_Sim_Len_Logistic':acc_logistic_tfidf_sim_len,'TFIDF_Cos_Sim_Len_XGB':acc_xgboost_tfidf_sim_len,
                     'TFIDF_Cos_Sim_Len_Dist_Logistic':acc_logistic_tfidf_sim_len_dist,'TFIDF_Cos_Sim_Len_Dist_XGB':acc_xgboost_tfidf_sim_len_dist,
                     'TFIDF_Cos_Sim_Len_Dist_ExtraFeat_Logistic':acc_logistic_tfidf_sim_len_dist_extra,'TFIDF_Cos_Sim_Len_Dist_ExtraFeat_XGB':acc_xgboost_tfidf_sim_len_dist_extra,
                    }, index=['Train','Validation', 'Test'])
hist.to_csv('hist.csv')

In [349]:
hist


,BOW_No_Feat_Logistic,BOW_No_Feat_XGB,TFIDF_No_Feat_Logistic,TFIDF_No_Feat_XGB,TFIDF_Cos_Sim_Logistic,TFIDF_Cos_Sim_XGB,TFIDF_Cos_Sim_Len_Logistic,TFIDF_Cos_Sim_Len_XGB,TFIDF_Cos_Sim_Len_Dist_Logistic,TFIDF_Cos_Sim_Len_Dist_XGB,TFIDF_Cos_Sim_Len_Dist_ExtraFeat_Logistic,TFIDF_Cos_Sim_Len_Dist_ExtraFeat_XGB
Train,0.856002,0.865919,0.841687,0.907017,0.907497,0.976672,0.910557,0.989327,0.912255,0.991571,0.916096,0.993332
Validation,0.795393,0.809374,0.795824,0.823410,0.872678,0.898754,0.877148,0.906084,0.879433,0.909906,0.884337,0.916824
Test,0.795699,0.809778,0.797240,0.827243,0.877247,0.901926,0.881064,0.906457,0.882989,0.909504,0.888157,0.915805


At the end adding features we always improve: the biggest improvement after adding cosine similarity. The final best score in validation is 0.915805 with a tes